In [17]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline

In [4]:
#fix random seed for reproductability
seed = 7
np.random.seed(seed)

In [5]:
#load data
dataframe = pd.read_csv("sonar.data", header=None)
dataset = dataframe.values

In [19]:
#split dataset into input and label
X = dataset[:, 0:60].astype(float)
y = dataset[:,60]
#Encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)

In [26]:
#base line model
def create_baseline():
    #create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, init='normal', activation='relu'))
    model.add(Dense(30, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    #compile model
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [27]:
estimators = []
estimators.append(('sandaridize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn= create_baseline, nb_epoch=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" %(results.mean()*100, results.std()*100))

Baseline: 83.61% (6.24%)


In [28]:
def create_dropout():
    #create model
    model = Sequential()
    model.add(Dropout(0.2, input_shape=(60,)))
    model.add(Dense(60, input_dim=60, init='normal', activation='relu'))
    model.add(Dense(30, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    #compile model
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [29]:
estimators = []
estimators.append(('sandaridize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn= create_dropout, nb_epoch=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Dropout at Visible layer: %.2f%% (%.2f%%)" %(results.mean()*100, results.std()*100))

Baseline: 86.00% (5.60%)


In [32]:
def create_dropout_hidden():
    #create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, init='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(30, init='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    #compile model
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [33]:
estimators = []
estimators.append(('sandaridize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn= create_dropout_hidden, nb_epoch=300, batch_size=16, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_y, cv=kfold)
print("Dropout at Hidden layer: %.2f%% (%.2f%%)" %(results.mean()*100, results.std()*100))

Dropout at Hidden layer: 84.16% (4.73%)
